In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
# /opt/ml/code/mmdetection_trash/configs/trash/swin_t/cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py
cfg = Config.fromfile('/opt/ml/code/mmdetection_trash/configs/trash/swin_t/swin_htc_Jinhyoen.py')

PREFIX = '../../input/data/'


project_name = 'obj' ##프로젝트 이름
run_name = 'swin_base_htc_00_jh'; ##run 이름
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config
}

cfg.log_config.hooks[1].init_kwargs['name'] = run_name    # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list    # config

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.seg_prefix = PREFIX
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_data4.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.seg_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'valid_data4.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 2

#num classes
cfg.model.roi_head.bbox_head[0].num_classes = 11
cfg.model.roi_head.bbox_head[1].num_classes = 11
cfg.model.roi_head.bbox_head[2].num_classes = 11

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/swin_htc_0_jinhyoen'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.model.pretrained = '/opt/ml/code/mmdetection_trash/work_dirs/swin_b_cascade_rcnn/swin_base_patch4_window7_224_22k.pth'

#학습 중간부터 다시하는 방법
#cfg.resume_from = './work_dirs/swin_t_mask_rcnn/epoch_15.pth'

#cfg.runner.max_epochs = 20

In [3]:
model = build_detector(cfg.model)

2021-05-19 15:30:52,456 - mmdet - INFO - load model from: /opt/ml/code/mmdetection_trash/work_dirs/swin_b_cascade_rcnn/swin_base_patch4_window7_224_22k.pth
2021-05-19 15:30:53,721 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.43s)
creating index...
index created!
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2611: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-19 15:31:02,187 - mmdet - INFO - Start running, host: root@298450a5b067, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/swin_htc_0_jinhyoen
2021-05-19 15:31:02,188 - mmdet - INFO - workflow: [('train', 1)], max: 25 epochs
/opt/ml/code/mmdetection_trash/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=1.18s)
creating index...
index created!
wandb: Currently logged in as: aibc_sjw (use `wandb login --relogin` to force relogin)


2021-05-19 15:32:15,798 - mmdet - INFO - Epoch [1][50/1311]	lr: 4.910e-06, eta: 12:15:19, time: 1.348, data_time: 0.075, memory: 15586, loss_rpn_cls: 0.6822, loss_rpn_bbox: 0.0491, loss_semantic_seg: 3.0646, s0.loss_cls: 1.2062, s0.acc: 67.8750, s0.loss_bbox: 0.0048, s0.loss_mask: 4.5939, s1.loss_cls: 0.3635, s1.acc: 82.0898, s1.loss_bbox: 0.0008, s1.loss_mask: 2.7989, s2.loss_cls: 0.3240, s2.acc: 65.8672, s2.loss_bbox: 0.0001, s2.loss_mask: 2.3313, loss: 15.4194, grad_norm: 246.2735
2021-05-19 15:33:23,078 - mmdet - INFO - Epoch [1][100/1311]	lr: 9.909e-06, eta: 12:13:30, time: 1.346, data_time: 0.026, memory: 15586, loss_rpn_cls: 0.6146, loss_rpn_bbox: 0.0491, loss_semantic_seg: 0.9703, s0.loss_cls: 0.2038, s0.acc: 97.9199, s0.loss_bbox: 0.0152, s0.loss_mask: 1.2166, s1.loss_cls: 0.0935, s1.acc: 98.2734, s1.loss_bbox: 0.0046, s1.loss_mask: 0.7581, s2.loss_cls: 0.0457, s2.acc: 98.4121, s2.loss_bbox: 0.0007, s2.loss_mask: 0.4325, loss: 4.4048, grad_norm: 56.4123


KeyboardInterrupt: 